In [1]:
import mlflow
from pyspark.ml.regression import RandomForestRegressor, LinearRegression, GBTRegressor

from pyspark.ml import Pipeline
from pyspark.ml.feature import  StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.sql.functions import col, concat, lit, hour, dayofweek, round
        

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_name="modelling_duration_exp")

<Experiment: artifact_location='/home/abhishek-wsl/codes/MLops_project/final_codes/mlruns/1', creation_time=1693228766019, experiment_id='1', last_update_time=1693228766019, lifecycle_stage='active', name='modelling_duration_exp', tags={}>

In [3]:
# class RegressorWithProcessing(mlflow.pyfunc.PythonModel):
#     def __init__(self, params, model=None) -> None:
#         from pyspark.sql import SparkSession
#         self.params = params
#         self.lr_model = model
#         # self.indexer_final = indexer_final
#         # self.encoder_final = encoder_final
#         # self.spark = SparkSession.builder.appName('prod_model').getOrCreate()
    
#     def read_process_df(self, data_path):
#         from pyspark.sql.functions import col, concat, lit, hour, dayofweek, round
#         from pyspark.sql import SparkSession
#         spark = SparkSession.builder.appName('prod_model').config("spark.driver.memory", "2g").config("spark.executor.memory", "6g")\
#     .config('spark.executor.cores',2).config('spark.default.parallelism',4).config("spark.jars.packages", "org.mlflow:mlflow-spark:2.2.0").getOrCreate()

#         df = spark.read.format('parquet').load(data_path)
#         df = df.select('VendorID','lpep_pickup_datetime','lpep_dropoff_datetime','PULocationID','DOLocationID','trip_distance')
#         df = df.withColumn('duration',\
#             round((col('lpep_dropoff_datetime')-col('lpep_pickup_datetime'))\
#             .cast("long")/60,2))
#         df = df.filter(col('duration')>=0.05).filter(col('duration')<=82)
#         df = df.withColumn('PU_DO',concat(col('PULocationID'),lit('_'),col('DOLocationID')))
#         df = df.withColumn('pu_hour',hour(col('lpep_pickup_datetime')))
#         df = df.withColumn('pu_weekday',dayofweek(col('lpep_pickup_datetime')))
        
#         df = df.select('VendorID','pu_hour','pu_weekday','PU_DO', 'trip_distance','duration')
#         # y = df.select('')

#         print(df.count(), len(df.columns))
#         return df

    
#     def prepare_data(self, data_path, categorical_cols, indexer_final=None, encoder_final=None, is_train=True):
#         from pyspark.ml.feature import  StringIndexer, OneHotEncoder, VectorAssembler
#         from pyspark.ml import Pipeline
#         df_processed = self.read_process_df(data_path)
#         indexers = [StringIndexer(inputCol=col,outputCol=col+'_index').fit(df_processed) for col in categorical_cols]
#         if not is_train:
#             df_processed = df_processed.dropna(subset='duration')
#             [indexer.setHandleInvalid("keep") for indexer in indexers]
#         indexer_pipeline = Pipeline(stages=indexers)
#         if indexer_final==None:
#             indexer_final = indexer_pipeline.fit(df_processed)
        
#         indexed_df = indexer_final.transform(df_processed)

#         encoder = [OneHotEncoder(inputCol=col+'_index',outputCol=col+'_onehot') for col in categorical_cols]
#         encoder_pipeline = Pipeline(stages = encoder)
#         if encoder_final==None:
#             encoder_final = encoder_pipeline.fit(indexed_df)
#         encoded_df = encoder_final.transform(indexed_df)
        
#         return encoded_df, indexer_final, encoder_final
    
#     def fit(self, data_path,feature_cols = ['trip_distance','VendorID_onehot','pu_hour_onehot','pu_weekday_onehot','PU_DO_onehot'],label_col='duration', categorical_cols = ['VendorID','pu_hour','pu_weekday','PU_DO'], regressor = LinearRegression,**kwargs):
#         from pyspark.ml.feature import VectorAssembler
#         from pyspark.ml import Pipeline
#         encoded_df, indexer_final, encoder_final = self.prepare_data(data_path,categorical_cols)
#         assembler = VectorAssembler(inputCols = feature_cols, outputCol = 'features')
#         regressor = regressor(featuresCol = 'features', labelCol= label_col,**self.params,**kwargs )
#         pipeline = Pipeline(stages = [assembler,regressor])
#         model = pipeline.fit(encoded_df)
#         self.model = model
#         return model, indexer_final, encoder_final

#     def predict(self, data_path, indexer_final=None, encoder_final=None, categorical_cols = ['VendorID','pu_hour','pu_weekday','PU_DO']):
#         encoded_df, _, _ = self.prepare_data(data_path,categorical_cols, indexer_final, encoder_final,is_train=False)
#         self.predictions = self.model.transform(encoded_df)
#         return self.predictions.select('prediction')
    
    


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('prod_model').config("spark.driver.memory", "2g").config("spark.executor.memory", "6g")\
    .config('spark.executor.cores', 2).config('spark.default.parallelism', 4).config("spark.jars.packages", "org.mlflow:mlflow-spark:2.2.0").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/09/10 18:34:17 WARN Utils: Your hostname, Bhaiyu resolves to a loopback address: 127.0.1.1; using 172.17.120.207 instead (on interface eth0)
23/09/10 18:34:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/abhishek-wsl/miniconda3/envs/mlops/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/abhishek-wsl/.ivy2/cache
The jars for the packages stored in: /home/abhishek-wsl/.ivy2/jars
org.mlflow#mlflow-spark added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1f19fdd7-8efd-47f9-90fd-7225100ef14d;1.0
	confs: [default]
	found org.mlflow#mlflow-spark;2.2.0 in central
	found org.slf4j#slf4j-api;1.7.25 in central
:: resolution report :: resolve 145ms :: artifacts dl 5ms
	:: modules in use:
	org.mlflow#mlflow-spark;2.2.0 from central in [default]
	org.slf4j#slf4j-api;1.7.25 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-------------------------------------------------------------------

In [5]:
spark.sparkContext.setLogLevel("ERROR")

In [6]:
class RegressorWithProcessing(mlflow.pyfunc.PythonModel):
    def __init__(self, params, model=None) -> None:
        self.params = params
        self.lr_model = model
        self.indexer_final = None
        self.encoder_final = None

    def read_process_df(self, df):
        # from pyspark.sql.functions import col, concat, lit, hour, dayofweek, round
        

        #df = spark.read.format('parquet').load(data_path)
        df = df.select('VendorID','lpep_pickup_datetime','lpep_dropoff_datetime','PULocationID','DOLocationID','trip_distance')
        df = df.withColumn('duration',\
            round((col('lpep_dropoff_datetime')-col('lpep_pickup_datetime'))\
            .cast("long")/60,2))
        df = df.filter(col('duration')>=0.05).filter(col('duration')<=82)
        df = df.withColumn('PU_DO',concat(col('PULocationID'),lit('_'),col('DOLocationID')))
        df = df.withColumn('pu_hour',hour(col('lpep_pickup_datetime')))
        df = df.withColumn('pu_weekday',dayofweek(col('lpep_pickup_datetime')))

        df = df.select('VendorID','pu_hour','pu_weekday','PU_DO', 'trip_distance','duration')
        print(df.count(), len(df.columns))
        return df

    def prepare_data(self, data_path, categorical_cols,is_train=True):
        # from pyspark.ml.feature import  StringIndexer, OneHotEncoder, VectorAssembler
        # from pyspark.ml import Pipeline
        df_processed = self.read_process_df(data_path)
        indexers = [StringIndexer(inputCol=col, outputCol=col+'_index').fit(df_processed) for col in categorical_cols]
        if not is_train:
            df_processed = df_processed.dropna(subset='duration')
            [indexer.setHandleInvalid("keep") for indexer in indexers]
        indexer_pipeline = Pipeline(stages=indexers)
        if self.indexer_final is None:
            self.indexer_final = indexer_pipeline.fit(df_processed)

        indexed_df = self.indexer_final.transform(df_processed)

        encoder = [OneHotEncoder(inputCol=col+'_index', outputCol=col+'_onehot') for col in categorical_cols]
        encoder_pipeline = Pipeline(stages=encoder)
        if self.encoder_final is None:
            self.encoder_final = encoder_pipeline.fit(indexed_df)

        encoded_df = self.encoder_final.transform(indexed_df)

        return encoded_df

    def fit(self, data_path, feature_cols=['trip_distance','VendorID_onehot','pu_hour_onehot','pu_weekday_onehot','PU_DO_onehot'], label_col='duration', categorical_cols=['VendorID','pu_hour','pu_weekday','PU_DO'], regressor=LinearRegression, **kwargs):
        # from pyspark.ml.feature import VectorAssembler
        # from pyspark.ml import Pipeline
        encoded_df = self.prepare_data(data_path, categorical_cols)
        assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
        regressor = regressor(featuresCol='features', labelCol=label_col, **self.params, **kwargs)
        pipeline = Pipeline(stages=[assembler, regressor])
        model = pipeline.fit(encoded_df)
        self.model = model
        return model

    def predict(self, data_path, categorical_cols=['VendorID','pu_hour','pu_weekday','PU_DO']):
        encoded_df = self.prepare_data(data_path, categorical_cols, is_train=False)
        self.predictions = self.model.transform(encoded_df)
        return self.predictions.select('prediction')


In [7]:
params = {'elasticNetParam':	0.07666740311526124,
            'fitIntercept':	True,
            'regParam':	0.1452221778539822}

model_obj = RegressorWithProcessing(params)

In [8]:
model_obj.params

{'elasticNetParam': 0.07666740311526124,
 'fitIntercept': True,
 'regParam': 0.1452221778539822}

In [9]:
df = spark.read.format('parquet').load('/home/abhishek-wsl/codes/MLops_project/data/*.parquet')
_ = model_obj.fit(df)

333262 6


In [10]:
predictions = model_obj.predict(df)

333262 6


In [11]:
predictions

DataFrame[prediction: double]

In [12]:
# df_1 = model.spark.read.format('parquet').load('/home/abhishek-wsl/codes/MLops_project/data/test_data/green_tripdata_2023-01.parquet')
# df_1 = df_1.limit(5)

In [13]:
# from mlflow.models.signature import infer_signature

# signature = infer_signature(df_1, predictions)

In [14]:
spark.stop()

In [15]:
#using user defined python class containing pyspark preprocessing and ml model from mlflow.pyfunc.PythonMode
with mlflow.start_run(run_name='lr_model_test1') as run:
    mlflow.pyfunc.log_model(
        'lr_preprocessed_model',
        python_model=model_obj
        # artifacts=artifacts,
        # conda_env= conda_env,
        # signature = signature,
        # input_example=df_1.toPandas()
    )
    # mlflow.spark.log_model(indexer_final,artifact_path='indexer')
    # mlflow.spark.log_model(encoder_final,artifact_path='encoder')

TypeError: cannot pickle '_thread.RLock' object

In [16]:
import dill

serialized_object = dill.dumps(model_obj)
print("Serialization successful.")
# except Exception as e:
#     print(f"Serialization failed: {e}")

TypeError: cannot pickle 'socket' object